In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
np.set_printoptions(legacy = '1.25')

import diem
import time

import sys
import os

# Prepare Data for `diem` analysis

## Convert vcf file to diem input files

In [ ]:
vcfFile = './data/example_dataset.vcf.gz'

In [ ]:
os.system('vcf2diem '+vcfFile)

## Update Ploidy information

vcf2diem generates a meta data file that provides information on chromosome lengths as well as the ploidy of each chromosome for each individual. 

By default, it assigns a ploidy of 2 across all individuals and chromosomes.  However, for some chromosomes, this will not be the case.  For example, in an XY system, the male is haploid for the X chromosome.  

In our example, we provide a file to update and correct this information.

We apply this update using the update_ploidy function

In [ ]:
metaRawFile = './data/example_dataset.vcf.gz.diem_meta.bed'

ploidyFile = './user_files/chrom_ploidy.tsv'

metaCorrected = './data/example_dataset.vcf.gz.diem_meta_corrected.bed'

In [ ]:
diem.update_ploidy(ploidyFile,metaFilePathIn=metaRawFile,metaFilePathOut=metaCorrected)

# Reading data into diemPy

We now have our inbut bed file and our *corrected* meta data file.  

We can read that in as a 'raw' or 'unpolarized' DiemType object.

The DiemType is a class which holds the data and provides the core functionality of diem. Refer to documentation for full details.

Here, we will simply show how it is used in a standard workflow. 

In [ ]:
bedInFile = './data/example_dataset.vcf.gz.diem_input.bed'
metaCorrected = './data/example_dataset.vcf.gz.diem_meta_corrected.bed'


In [ ]:
dRaw = diem.read_diem_bed(bedInFile,metaCorrected)


Let's have a look at the variables (':ivar') that store the information for diem:

In [ ]:
?dRaw

# Polarization

To polarize the data, we simply call the polarize() function of the DiemType object we created above.

Note:
- that it defaults to a parallel evaluation using the full number of available cores. We can change the number of cores manually using the 'ncores=' optional argument.  If n=1, it performs a serial polarization of the data.
- it returns a separate *copy* of the DiemType object that is now polarized. That is, the raw object is not changed.

In [ ]:
dPol = dRaw.polarize(ncores=4,epsilon = 0.999999999999) 

We can now save this to a bed file which has the same format as the input file but added information from polarization

In [ ]:
bedOutFile = './output/diem_output.bed'
diem.write_polarized_bed(bedInFile,bedOutFile,dPol)


We can also save this directly as a diemtype object. 

It uses less memory, and it is how we will work with the data moving forward.

However, it is only useable within the diem environment. 

In [ ]:
polarizedDiemTypeOutFile = './output/example_polarized.diemtype'

diem.save_DiemType(dPol,polarizedDiemTypeOutFile)


If we have already polarized the data, we can load the data back in either way:

In [ ]:
dPol = diem.load_DiemType(polarizedDiemTypeOutFile) 
#or
dPol = diem.read_diem_bed(bedOutFile,metaCorrected)

# Hybrid Indices and Sorting 

the `diem` algorithm sorts individuals by hybrid index and seeks to maximize the differences between two sets of individuals on either side of a barrier to gene flow.

although the hybrid indices are calculated on polarization, the individuals are not sorted by hybrid index as we can see here:

In [ ]:
plt.figure(figsize=(5,4))
plt.plot(dPol.HIs,marker='.',)
plt.ylim(0,1)
plt.show()

We can use the sort() function to sort the individuals. 

In [ ]:
dPol.sort()

In [ ]:
plt.figure(figsize=(5,4))
plt.plot(dPol.HIs,marker='.',)
plt.ylim(0,1)
plt.show()

Now that we have the 

# Plotting a Painting

Now that we have polarized and sorted the data, let's have a look at it using the plot_painting() function:

In [ ]:
#here we are plotting chromosome 9
diem.plot_painting(dPol.DMBC[9])


Simply for illustration, let's compare this to the null polarization that diem started with:

In [ ]:
diem.plot_painting(diem.diemtype.flip_polarity(dPol.DMBC[9],dPol.initialPolByChr[9]))

# The Diagnostic Index (DI) and Thresholding

## The DI distribution 

Here we plot the distribution of diagnostic indices across the data.  For each of the final marker configurations, the diagnostic index tells use how well the marker matches does at explaining our data.  The least-negative value is the best.

In teal, we plot a histogram of the DI values for each unique marker.

In magenta, the counts of each marker are taken into account.

In [ ]:
DIValsWithRepeats = np.hstack(dPol.DIByChr)
DIValsUnique = np.unique(DIValsWithRepeats)
plt.figure(figsize=(5,4))
(counts,bins,patches) = plt.hist(DIValsWithRepeats,bins=500,color='darkmagenta')
plt.hist(DIValsUnique,bins=bins,color='lightseagreen')
plt.yscale('log')
plt.show()

We provide a function that will characterize the marker configurations, and provide information about each:  the count, the DI, the number of differences from the 'ideal marker' (the one with the highest DI) and how  many of the individuals have missing data for that marker. It returns a pandas dataframe.

This could be useful for exploring the data in greater detail when deciding on a threshold value in later steps.

**note** that this could be memory and time intensive to generate if there are many marker types.  We recommend characterizing only a subset of the most-informative markers to be on the safe side.  It is those markers that we will retain through thresholding later.



In [ ]:
dfMarkers = diem.characterize_markers(dPol.apply_threshold(-70))
dfMarkers

## applying the final threshold chosen

The DI distribution for every dataset will be unique. However, many of the markers in the data are SNPs that are not informative about the barrier.

For downstream analysis, we want to choose a threshold for retaining markers.  

Here, we pick a threshold of -70.  Thresholding returns a copy of the data, and the propoertion of markers retained for each chromsoome is reported

In [ ]:
dThresh = dPol.apply_threshold(-60)

After thresholding, the hybrid indices may change.  We can update these and re-sort the individuals in the data using the sort() function again:

In [ ]:
dThresh.sort()

Let's compare the before (b) and after (r) thresholding:

In [ ]:
plt.figure(figsize=(5,4))
plt.plot(dPol.HIs,marker='.',color='b')
plt.plot(dThresh.HIs,marker='.',color='r')
plt.ylim(0,1)
plt.show()

After thresholding, there will be fewer markers present. 
Let's plot the painting after thresholding and see how much cleaner it looks. 

In [ ]:
diem.plot_painting(dThresh.DMBC[9])

# Smoothing the Data

After thresholding, we will see much clearer tracts representing shared ancestry with respect to the barrier.  However, there will be some bits of missing data, mis-called sites, etc. that will break up the tracts. We want to smooth this over for later analyses. We also care about tracts generated by cross-over recombination, but gene conversion events could also cause short changes in state and break up our tracts.  Therefore, we want to smooth over this noise and this signal in the data.

To pick a smoothing scale, we examine how the data changes for different values.  Below, we calculate the number of sites that change across the whole dataset using the laplacian smoothign kernel:

In [ ]:
scalesToTry = [1e-8,5e-8,1e-7,5e-7,1e-6,5e-6,1e-5,5e-5,1e-4,5e-4]
sitesDiffByScale = []
for scale in scalesToTry:
    dSmoothedTest = dThresh.smooth(scale)
    kdiffs = diem.count_site_differences(dSmoothedTest.DMBC,dThresh.DMBC)
    sitesDiffByScale.append(kdiffs)

In [ ]:
plt.figure(figsize=(5,4))
plt.plot(scalesToTry,sitesDiffByScale,marker='.')
plt.show()

We see that by around scale = 5e-4, the single-site changes and short tracts have been smoothed over, and we choose this as our final smoothing scale.

**note** smoothing returns a copy of the dataset with the same order (i.e. HIs are not recomputed and data is not resorted). That way, comparisons can be made easily before and after smoothing. 


In [ ]:
dSmoothed = dThresh.smooth(5e-5)

We can see how smoothing has affected our HIs

In [ ]:
plt.figure(figsize=(5,4))
plt.plot(dThresh.HIs,marker='.',color='b')
plt.plot(dSmoothed.computeHIs(),marker='.',color='r')
plt.show()

 and we can plot the effect of smoothing on the data:

In [ ]:
diem.plot_painting(dSmoothed.DMBC[9])

# Constructing Contigs

Downstream analyses of diem output may focus on the lengths of tracts of shared ancestry across the gene flow barrier.

Here, we provide the function create_contig_matrix() that generates 'contigs' for each chromosome of each individual.

a 'contig' is a summary of the data.  It is the sequence of contiguous intervals that form a givien chromosome 

Once we have created the contigs, we can export per-individual contig files in bed formatting 

In [ ]:
dSmoothed.create_contig_matrix()


In [ ]:
contingOutputDirectory = './output/contigs/'

diem.export_contigs_to_ind_bed_files(dSmoothed,contingOutputDirectory)


# Using Contigs & intervals

Within the diemtype objects, contigs are stored in the *contigMatrix*

the *contigMatrix* is an array wit shape nChrs x nIndividuals. and we can use splicing to access specific contigs. 

For chromosome indexed 1 and individual indexed 8:

In [ ]:
exampleContig = dSmoothed.contigMatrix[1,8]
exampleContig.__dict__

A contig stores the individual and chromosome names to which it belongs as well as a list of intervals and the number of intervals

In [ ]:
?exampleContig

In [ ]:
exampleContig.num_intervals

an **interval** is a contiguous segment of genome with the same ancestry.

It stores the chromosome and individual to which it belongs. the indices and positions of the left- and right-most markers,
and the state of the individual: 0, 1, 2, or 3.

In [ ]:
exampleInterval = exampleContig.intervals[0]
?exampleInterval

In [ ]:
exampleInterval.__dict__

We can get the interval's span easily:

In [ ]:
exampleInterval.span()

And we can get the intervals map span (on a unit scale) if we provide the length of the chromsome.

In [ ]:
exampleInterval.mapSpan(dSmoothed.chrLengths[1])

Here is the information about all the intervals that form the genotype of our example contig:

In [ ]:
[x.__dict__ for x in exampleContig.intervals]

We can plot the tract length distribution for all intervals with state '1'

# Plotting tract length distributions

In [ ]:
oneTracts = dSmoothed.get_intervals_of_state(1)
oneLengths = [x.span() for x in oneTracts]
plt.hist(oneLengths,color='m')
plt.show()

We can plot the distribution of the different types of tracts together

In [ ]:
plt.hist([x.span() for x in dSmoothed.get_intervals_of_state(1)],bins = 50,color='m')
plt.hist([x.span() for x in dSmoothed.get_intervals_of_state(2)],bins=50,color='y')
plt.hist([x.span() for x in dSmoothed.get_intervals_of_state(3)],bins=50,color='teal')
plt.show()

# Advanced tract length distribution plotting

We can also subset individuals and chromosomes when getting intervavls of those types.

the barrier splits the population at the steepest change in HI. We can get the index which splits the barrier in two as follows:

In [ ]:
nIndsLeft = np.argmax(dSmoothed.HIs[1:] - dSmoothed.HIs[0:-1])+1
print(nIndsLeft)

In [ ]:
leftIndividuals = np.arange(0,nIndsLeft)
rightIndividuals = np.arange(nIndsLeft,len(dSmoothed.indNames))
print(leftIndividuals)
print(rightIndividuals)

We can plot the spans of tracts of type '1' for individuals on the 'left' (blue) and 'right' (red) sides of the barrier

In [ ]:
state1SpansLeftSide = [x.span() for x in dSmoothed.get_intervals_of_state(1,individualSubset=leftIndividuals)]
state1SpansRightSide = [x.span() for x in dSmoothed.get_intervals_of_state(1,individualSubset=rightIndividuals)]

plt.hist(state1SpansLeftSide,bins = 100, color = 'b',alpha = 0.5)
plt.hist(state1SpansRightSide, bins = 100, color = 'r',alpha = 0.5)
plt.show()

Similarly, we can plot the spans of type '3' comparing the left and right sides of the barrier

In [ ]:
state3SpansLeftSide = [x.span() for x in dSmoothed.get_intervals_of_state(3,individualSubset=leftIndividuals)]
state3SpansRightSide = [x.span() for x in dSmoothed.get_intervals_of_state(3,individualSubset=rightIndividuals)]

plt.hist(state3SpansRightSide, bins = 100, color = 'r',alpha = 0.5)
plt.hist(state3SpansLeftSide,bins = 100, color = 'b',alpha = 0.5)
plt.show()

# Plotting paintings with marker spacing

We have provided a function which plots the paintings but accounts for the physical spacing between markers. Note that the resolution may be too poor on a dense dataset to see the difference.


In [ ]:
diem.plot_painting(dSmoothed.DMBC[9])
diem.plot_painting_with_positions(dSmoothed.DMBC[9],dSmoothed.posByChr[9])


# Advanced Polarizing

## Site and Individual Masks

When running diem, we may want to mask some individual and sites.  Here, masked sites/individuals do not *influence* the barrier (i.e. they don't contribute to the likelihood function). However, they *are* polarized during the EM step along with other sites and individuals. 

We have included two example exclusion files under the user_files subdirectory.

Here, we mask individuals that were sampled outside of france and spain.

We also (arbitrarily) mask all of chromosome 2 for this example as well as two single sites on chromosome 3

Although we have masked both sites and individuals in this example, it is not necessary to specify both. Any masks have been added to the raw data will be used in polarization

In [ ]:
# #### add information about individuals that do not influence polarization. We say 'excluded' but really mean 'masked'
sitesMaskedFile = './user_files/masked_regions.bed'
individualsMaskedFile = './user_files/masked_individuals.txt'

dRawWithMasks = dRaw.copy()

dRawWithMasks.add_individual_exclusions(individualsMaskedFile)
dRawWithMasks.add_site_exclusions(sitesMaskedFile)

In [ ]:
dPolWithMasks = dRawWithMasks.polarize()
dPolWithMasks.sort()

In [ ]:
diem.plot_painting(dPol.DMBC[9])


In [ ]:
plt.figure(figsize=(5,4))
plt.plot(dPolWithMasks.HIs,marker='.',color='b')
plt.show()

## Specifying a pre-defined null polarity

In [ ]:
dRawPredefinedInitial = diem.read_diem_bed(bedInFile,metaCorrected)
dRawPredefinedInitial.add_initial_polarity('./user_files/user_initial_pol.txt')

dPolPredefinedInitial = dRawPredefinedInitial.polarize(ncores=4)